In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
def sqrt_func(i, j):
    time.sleep(1)
    return i + j


Parallel(n_jobs=2)(delayed(sqrt_func)(i, 5)
                   for i in range(5))


[5, 6, 7, 8, 9]

## Functions

In [3]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

96


In [4]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [5]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [6]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


982


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [7]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.94644416e-08, 2.88473643e-08, 1.32387770e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 981)

In [8]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [9]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


314.4567873998194

In [10]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [11]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results['objVal']

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [3 0 3 3 2 4 2 4 3 1 4 0 1 4 1 2 2 2 1 1 2 0 0 4 2 1 4 2 4 0 0 3 1 3 4 4 2
 1 1 1 1 2 0 1 3 4 2 1],
N: 96,
Objective value: 372.93271718042274,
Prob calculation time: 17.742953 sec,
Waiting time (timer): 314.4567873998194 (36.824536 sec),
Idle time (timer): -4.083329632508367 (0.000002 sec),
Tardiness (timer): 619.9166703674916 (0.000334 sec)



372.93271718042274

In [12]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([2, 8, 9, 7, 8]),
 [0.0, 0.25, 0.2916666666666667, 0.20833333333333331, 0.25],
 1.0,
 [0.6363636363636362,
  0.09090909090909093,
  0.0,
  0.18181818181818185,
  0.09090909090909093],
 1.0)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [13]:
# OLD
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        child1[i] += -1
        child2[i] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        child2[i] += -1
        child1[i] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[1 4 4 4 4 2 0 3 4 0] [2 4 3 3 0 0 2 4 2 3]


(array([1, 3, 4, 4, 3, 2, 1, 3, 3, 2]), array([2, 4, 2, 2, 0, 0, 4, 4, 3, 2]))

In [ ]:
# NEW
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    p2 = createDist(child2, reverse=True)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        j = np.random.choice(I, 1, p=p2)
        child1[i] += -1
        child2[j] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    p1 = createDist(child1, reverse=True)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        j = np.random.choice(I, 1, p=p1)
        child2[i] += -1
        child1[j] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[1 4 4 4 4 2 0 3 4 0] [2 4 3 3 0 0 2 4 2 3]


(array([1, 3, 4, 4, 3, 2, 1, 3, 3, 2]), array([2, 4, 2, 2, 0, 0, 4, 4, 3, 2]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [14]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[1 3 4 4 3 2 1 3 3 2]
[1 4 3 4 3 2 1 3 3 2]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [15]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((96, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [16]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1

    def create_bw_schedule(self):
        self.x = np.ones(self.I).astype(int)
        self.x[0] = 2
        checkSum = np.sum(self.x)
        print(f'checkSum = {checkSum}')
        diff = checkSum - self.N
        print(f'diff = {diff}')
        if diff > 0:
            i = np.random.choice(range(1, self.I), diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] - 1
        if diff < 0:
            i = np.random.choice(range(1, self.I), -diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
[3 3 3 2 2 0 1 3 0 2 3 2 3 4 3 1 4 2 1 3 3 1 0 1 2 3 1 2 2 2 2 3 0 0 2 2 3
 3 2 3 2 2 0 3 3 1 3 0]
96
checkSum = 49
diff = -47
i = [26 32 43  4 29 18 33 37 27 45 30 25 39  7 21 40 23  5 36 47 22  1  8 24
  9 44 11 28 31 38  6 19 35  3 46  2 42 20 16 15 12 13 41 34 17 14 10]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2]
96


Create population of individuals

In [17]:
# OLD
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in self.individuals])
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show,
                                            I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in newIndividuals])
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


In [18]:
#NEW
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array(Parallel(n_jobs=6)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in self.individuals))
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array(Parallel(n_jobs=6)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v,
                                                                              N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in newIndividuals))
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [19]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [1 0 1 1 1 3 2 2 1 0],
N: 12,
Objective value: 31.7946719902013,
Prob calculation time: 0.117662 sec,
Waiting time (timer): 27.00032862691508 (0.095554 sec),
Idle time (timer): 1.789015353487514 (0.000003 sec),
Tardiness (timer): 59.78901535348752 (0.000126 sec)

Schedule: [0 2 2 1 3 0 0 1 2 1],
N: 12,
Objective value: 36.066667863968455,
Prob calculation time: 0.117432 sec,
Waiting time (timer): 32.39540300278764 (0.109846 sec),
Idle time (timer): 4.937932725149267 (0.000003 sec),
Tardiness (timer): 62.937932725149274 (0.000141 sec)

Schedule: [2 2 1 1 0 1 3 2 0 0],
N: 12,
Objective value: 32.943858351516106,
Prob calculation time: 0.116264 sec,
Waiting time (timer): 31.145768517245898 (0.115859 sec),
Idle time (timer): -0.05805181421368388 (0.000002 sec),
Tardiness (timer): 57.94194818578631 (0.000062 sec)

Schedule: [1 1 2 0 0 0 3 1 2 2],
N: 12,
Objective value: 29.211593440276268,
Prob calculation time: 0.124798 sec,
Waiting time (timer): 23.176018519942996 (0.110779 sec)

In [20]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([2, 2, 1, 1, 0, 1, 3, 2, 0, 0]), array([0, 2, 2, 1, 3, 0, 0, 1, 2, 1]), array([1, 0, 1, 1, 1, 3, 2, 2, 1, 0]), array([1, 0, 0, 2, 4, 0, 3, 0, 0, 2]), array([1, 1, 2, 0, 0, 0, 3, 1, 2, 2])] 
 [array([2, 2, 1, 1, 0, 1, 3, 2, 0, 0]), array([0, 2, 2, 1, 3, 0, 0, 1, 2, 1]), array([1, 0, 1, 1, 1, 3, 2, 2, 1, 0]), array([1, 0, 0, 2, 4, 0, 3, 0, 0, 2]), array([1, 1, 2, 0, 0, 0, 3, 1, 2, 2])]


In [21]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [1 0 1 1 1 3 2 2 1 0],
N: 12,
Objective value: 31.7946719902013,
Prob calculation time: 0.110121 sec,
Waiting time (timer): 27.00032862691508 (0.083397 sec),
Idle time (timer): 1.789015353487514 (0.000002 sec),
Tardiness (timer): 59.78901535348752 (0.000069 sec)

Schedule: [2 2 1 1 0 1 3 2 0 0],
N: 12,
Objective value: 32.943858351516106,
Prob calculation time: 0.109597 sec,
Waiting time (timer): 31.145768517245898 (0.107027 sec),
Idle time (timer): -0.05805181421368388 (0.000002 sec),
Tardiness (timer): 57.94194818578631 (0.000065 sec)

Schedule: [0 2 2 1 3 0 0 1 2 1],
N: 12,
Objective value: 36.066667863968455,
Prob calculation time: 0.109459 sec,
Waiting time (timer): 32.39540300278764 (0.108116 sec),
Idle time (timer): 4.937932725149267 (0.000002 sec),
Tardiness (timer): 62.937932725149274 (0.000063 sec)

Schedule: [1 1 2 0 0 0 3 1 2 2],
N: 12,
Objective value: 29.211593440276268,
Prob calculation time: 0.107505 sec,
Waiting time (timer): 23.176018519942996 (0.109732 sec)

[array([1, 1, 1, 1, 0, 0, 2, 2, 1, 3]),
 array([1, 1, 2, 0, 0, 0, 2, 1, 3, 2]),
 array([1, 1, 2, 0, 0, 0, 2, 2, 2, 2]),
 array([2, 1, 1, 1, 1, 1, 1, 1, 2, 1]),
 array([1, 0, 2, 1, 1, 2, 1, 1, 1, 2])]

In [22]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [2 1 1 1 1 2 2 0 1 1],
N: 12,
Objective value: 31.9027929411266,
Prob calculation time: 0.106819 sec,
Waiting time (timer): 29.063537655363124 (0.079305 sec),
Idle time (timer): -0.05795177310992017 (0.000010 sec),
Tardiness (timer): 57.94204822689007 (0.000101 sec)

Schedule: [1 1 3 0 1 1 2 0 1 2],
N: 12,
Objective value: 30.95393893915395,
Prob calculation time: 0.113483 sec,
Waiting time (timer): 27.068926743305383 (0.097634 sec),
Idle time (timer): 0.03895113500252023 (0.000002 sec),
Tardiness (timer): 58.03895113500251 (0.000059 sec)

Schedule: [1 2 0 1 1 1 3 0 2 1],
N: 12,
Objective value: 30.321964047199153,
Prob calculation time: 0.106046 sec,
Waiting time (timer): 25.81240400683818 (0.092684 sec),
Idle time (timer): 0.03152408756012903 (0.000033 sec),
Tardiness (timer): 58.03152408756013 (0.000153 sec)

Schedule: [0 2 1 2 3 1 1 1 1 0],
N: 12,
Objective value: 37.10767067855244,
Prob calculation time: 0.106814 sec,
Waiting time (timer): 34.477340184087176 (0.091286 se

In [23]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,29.704842,"[1, 1, 2, 1, 0, 1, 1, 1, 2, 2]"
1,29.920887,"[1, 0, 1, 1, 1, 2, 0, 2, 2, 2]"
2,28.941279,"[1, 1, 0, 1, 0, 1, 2, 2, 2, 2]"


In [24]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 2 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 2
 0 0 0 1 1 1 0 1 1 1 0],
N: 24,
Objective value: 18.825283100360735,
Prob calculation time: 1.726449 sec,
Waiting time (timer): 8.07403373829789 (0.167462 sec),
Idle time (timer): 41.99278267506929 (0.000002 sec),
Tardiness (timer): 17.992782675069297 (0.000165 sec)

Schedule: [0 0 0 0 1 0 1 0 2 1 1 0 0 0 1 0 0 2 0 1 1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 2
 1 0 1 1 0 1 0 0 0 0 2],
N: 24,
Objective value: 18.60698241821808,
Prob calculation time: 1.699203 sec,
Waiting time (timer): 10.502412552497093 (0.328409 sec),
Idle time (timer): 40.01002899536542 (0.000002 sec),
Tardiness (timer): 16.010028995365392 (0.000265 sec)

Schedule: [0 1 0 0 0 1 1 1 1 2 1 1 2 0 0 0 0 0 0 0 0 0 1 3 1 0 1 2 0 0 0 0 0 0 0 1 1
 0 0 0 0 1 0 0 1 0 0 1],
N: 24,
Objective value: 15.975589273304923,
Prob calculation time: 1.721118 sec,
Waiting time (timer): 19.375886454143355 (0.376916 sec),
Idle time (timer): 29.70872448607929 (0.00000

In [25]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [26]:
scheduleObject = Schedule(dataObject)
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
x = scheduleObject.get_schedule()
calcFitness(x, beta, precision, limit, v, N, no_show,
            I, d, eind, alpha_W, alpha_I, alpha_T)


checkSum = 49
diff = 25
i = [22 28 24 35 17 26 41 14 34  6 11 21 29 37 31  1 23  7 47 32 19 25 16 27
 30]
[2 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 1 1 1 0 1 1 1 1 1 0]
24
Schedule: [2 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 1 1 1 0 1 1 1 1 1 0],
N: 24,
Objective value: 23.888718307251743,
Prob calculation time: 18.958119 sec,
Waiting time (timer): 13.304101634568829 (0.965065 sec),
Idle time (timer): 46.94512942237367 (0.000003 sec),
Tardiness (timer): 22.94512942237369 (0.000415 sec)



23.888718307251743

In [27]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 0 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0
 3 0 0 1 0 0 1 1 1 0 0],
N: 24,
Objective value: 15.04938919728676,
Prob calculation time: 1.638108 sec,
Waiting time (timer): 12.133433303092387 (0.127692 sec),
Idle time (timer): 32.993359793416346 (0.000002 sec),
Tardiness (timer): 8.993359793416344 (0.000113 sec)

Schedule: [0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 2 1 0 1 0 0 0 1 1 0 1 0 0
 0 0 0 2 0 2 1 0 0 1 1],
N: 24,
Objective value: 25.762812678871583,
Prob calculation time: 1.651364 sec,
Waiting time (timer): 13.070230146436222 (0.249439 sec),
Idle time (timer): 50.22453436716182 (0.000002 sec),
Tardiness (timer): 26.224534367161816 (0.000104 sec)

Schedule: [0 0 1 1 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 1 0 0 1 2 2 2 0 0 0 0 1 1 0 1 0 1 1
 0 1 0 0 2 1 1 0 0 1 0],
N: 24,
Objective value: 32.01471625117887,
Prob calculation time: 1.640641 sec,
Waiting time (timer): 19.574239606075405 (0.408105 sec),
Idle time (timer): 56.3083673479145 (0.00000

In [28]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [29]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 2 1 0 1 2 0 1 0 0 0 1 1 0
 0 0 0 0 1 0 0 0 1 2 1],
N: 24,
Objective value: 26.94470855188071,
Prob calculation time: 1.684018 sec,
Waiting time (timer): 16.24711065433547 (0.255640 sec),
Idle time (timer): 50.07644048357753 (0.000002 sec),
Tardiness (timer): 26.07644048357753 (0.000099 sec)

Schedule: [0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 0 1 0 3 3 1
 0 1 0 0 1 0 1 0 0 2 0],
N: 24,
Objective value: 36.22362859098516,
Prob calculation time: 1.699595 sec,
Waiting time (timer): 17.40661513589819 (0.335393 sec),
Idle time (timer): 64.76830422770979 (0.000002 sec),
Tardiness (timer): 40.768304227709805 (0.000117 sec)

Schedule: [0 2 0 2 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 2 0 0 0 1 0 1 0 0 3 1 0 0 0
 0 1 0 1 0 0 0 1 1 1 1],
N: 24,
Objective value: 18.439946467307674,
Prob calculation time: 1.703856 sec,
Waiting time (timer): 8.15910313324299 (0.378350 sec),
Idle time (timer): 41.293842023350805 (0.000002 s

In [30]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [31]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 0 0 0 0 1 1 0 1 0 0 0 2 0 0 1 0 1 2 1 0 1 1 1 2 1 0 0 1 1 0 0 0 1 1 0
 0 1 0 0 0 1 1 0 0 1 0],
N: 24,
Objective value: 22.721480251528106,
Prob calculation time: 1.603475 sec,
Waiting time (timer): 20.43695720194412 (0.240178 sec),
Idle time (timer): 40.24449561791744 (0.000002 sec),
Tardiness (timer): 16.244495617917426 (0.000110 sec)

Schedule: [0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 1 1 1 0 0 2 1 2 0 0 0 0 0 0 1 1 0
 0 2 1 0 1 1 0 0 1 0 2],
N: 24,
Objective value: 33.3026793874037,
Prob calculation time: 1.595840 sec,
Waiting time (timer): 18.046110001041644 (0.316661 sec),
Idle time (timer): 59.4737256449784 (0.000002 sec),
Tardiness (timer): 35.47372564497841 (0.000109 sec)

Schedule: [1 2 2 0 0 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 2 0 0 0 2 0 0 0 0 0 0 1 0 0 1 0 1
 0 0 3 0 1 0 0 1 0 0 0],
N: 24,
Objective value: 10.774986987123793,
Prob calculation time: 1.599763 sec,
Waiting time (timer): 10.62261259626098 (0.483660 sec),
Idle time (timer): 26.87656991436566 (0.000002 s

In [32]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [33]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 1 1 2 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 0 4 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 1 0 0 1 0 1],
N: 24,
Objective value: 14.121026896453062,
Prob calculation time: 1.680768 sec,
Waiting time (timer): 15.026440126117178 (0.233415 sec),
Idle time (timer): 29.51741807667699 (0.000002 sec),
Tardiness (timer): 5.517418076676977 (0.000107 sec)

Schedule: [1 0 1 0 0 0 1 2 1 0 2 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 2 1 0 0 0 2 0 2 0 0 0
 0 0 0 1 0 0 0 0 1 1 0],
N: 24,
Objective value: 13.14974428365543,
Prob calculation time: 1.680161 sec,
Waiting time (timer): 14.194287364283147 (0.379243 sec),
Idle time (timer): 28.45338222990361 (0.000002 sec),
Tardiness (timer): 4.45338222990362 (0.000110 sec)

Schedule: [0 0 1 1 0 0 0 0 1 1 1 0 1 2 0 1 0 2 0 0 0 0 0 3 0 3 0 0 0 1 0 0 0 1 1 0 0
 0 0 0 0 2 1 0 1 0 0 0],
N: 24,
Objective value: 15.068142689693925,
Prob calculation time: 1.691122 sec,
Waiting time (timer): 16.446524616531203 (0.457862 sec),
Idle time (timer): 30.149221405135734 (0.00000

In [34]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkCyan')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")
